# SQL 작성

In [45]:
import sys
import os
import pandas as pd
from scipy import io
import cv2
from glob import glob
sys.path.append('../../src')
import DISData as DD
import json

## SQL 연결

In [62]:
doUT = DD.SQL()

host:  192.168.0.128
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [3]:
query1 = (f'''CREATE TABLE IF NOT EXISTS sviro_box_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(255) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(255) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(20) NOT NULL,
            box_xtl INT,
            box_ytl INT,
            box_xbr INT,
            box_ybr INT);''')

In [4]:
query2 = (f'''CREATE TABLE IF NOT EXISTS sviro_landmark_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(255) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(255) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(20) NOT NULL,
            landmark VARCHAR(255));''')

In [5]:
query1_1 = (f'''INSERT INTO sviro_box_info 
        (img_name, img_dir, img_format, img_width, img_height, color_space,
        label_name, label_dir, label_format,
        label, box_xtl, box_ytl, box_xbr, box_ybr)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')
query2_2 = (f'''INSERT INTO sviro_landmark_info 
        (img_name, img_dir, img_format, img_width, img_height, color_space,
        label_name, label_dir, label_format,
        label, landmark)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

In [6]:
txt_labels = {0:'empty', 1:'babyseat', 2:'childseat', 3:'person', 4:'stuff'}
person_labels = {0:'empty', 1:'baby', 2:'child', 3:'Adult'}
landmark_label = {0:'empty', 1:'baby', 2:'child', 3:'Adult', 4:'stuff', 5:'babyseat', 6:'childseat'}

In [7]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [8]:
try:
    doUT.create_table(query2)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [69]:
value_list1 = []
value_list2 = []
error_list = []

colors = ['RGB','IR']

for color_space in colors:
    label_path = f'Z:\\sviro\\data\\{color_space}\\labeldata'
    img_path = f'Z:\\sviro\\data\\{color_space}\\rawdata'
    label_ids = os.listdir(label_path)
    for label_id in label_ids:
        label_files = os.listdir(os.path.join(label_path,label_id))  
        for cnt, label_file in enumerate(label_files):
            label = os.path.join(label_path,label_id,label_file)
            label_name, label_format = os.path.splitext(os.path.basename(label))
            label_format = label_format.replace('.','')
            
            if label_format == 'txt':
                label = os.path.join(label_path,label_id,label_file)
                image = label.replace('labeldata','rawdata').replace('.txt','.png')
                img_dir = image
                img_name, img_format = os.path.splitext(os.path.basename(image))
                img_format = img_format.replace('.','')
                try:
                    image = cv2.imread(img_dir)
                except Exception as e:
                    error_list.append(label_dir)
                    print(label_dir, e)
                    continue
                img_width = image.shape[1]
                img_height = image.shape[0]
                label_dir = label
                label_name, label_format = os.path.splitext(os.path.basename(label))
                label_format = label_format.replace('.','')

                with open(label,'r') as f:
                    data = f.readlines()
                result = []
                for i in data:
                    label_num, box_xtl, box_ytl, box_xbr, box_ybr = map(int,i.split(','))
                    result.append([label_num, box_xtl, box_ytl, box_xbr, box_ybr])
                sorted_result = sorted(result, key=lambda x: x[1])
                for i, box in enumerate(sorted_result):
                    # print(box)
                    label_num, box_xtl, box_ytl, box_xbr, box_ybr = map(int,box)
                    if label_num == 3:
                        box_center = (box_xtl+box_xbr)/2

                        if sorted_result[i-1][1] <= box_center and sorted_result[i-1][3] >= box_center and sorted_result[i-1][0] == 1:
                            label = 'baby'
                        elif sorted_result[i-1][1] <= box_center and sorted_result[i-1][3] >= box_center and sorted_result[i-1][0] == 2:
                            label = 'child'
                        else :
                            if i+1 >= len(sorted_result):
                                label = 'adult'
                            elif sorted_result[i+1][1] <= box_center and sorted_result[i+1][3] >= box_center and sorted_result[i+1][0] == 1:
                                label = 'baby'
                            elif sorted_result[i+1][1] <= box_center and sorted_result[i+1][3] >= box_center and sorted_result[i+1][0] == 2:
                                label = 'child'
                            else:
                                label = 'adult'
                    else:
                        label = txt_labels[label_num]

                    img_dir = img_dir.replace('Z:','DataBase')
                    label_dir = label_dir.replace('Z:','DataBase')

                    values = (img_name, img_dir, img_format, img_width, img_height, color_space,
                    label_name, label_dir, label_format,
                    label, box_xtl, box_ytl, box_xbr, box_ybr)

                    value_list1.append(values)
                    if len(value_list1) >= 500:
                        print(value_list1[-1])
                        print(f'{cnt} / {len(label_files)}')
                        doUT.insert_dataset_values(query1_1, value_list1)
                        value_list1 = []

            elif label_format == 'json':
                label = os.path.join(label_path,label_id,label_file)
                image = label.replace('labeldata','rawdata').replace('_pose.json','.png')
                img_dir = image
                img_name, img_format = os.path.splitext(os.path.basename(image))
                img_format = img_format.replace('.','')
                try:
                    image = cv2.imread(img_dir)
                except Exception as e:
                    error_list.append(label_dir)
                    print(label_dir, e)
                    continue
                img_width = image.shape[1]
                img_height = image.shape[0]
                label_dir = label
                label_name, label_format = os.path.splitext(os.path.basename(label))
                label_format = label_format.replace('.','')

                with open(label,'r',encoding='utf-8') as f:
                    data = json.load(f)

                for i in data:
                    points = []
                    position = data[i]['position']
                    if position == 'left':
                        label_num = int(img_name.split('_')[-3])
                    elif position == 'middle':
                        label_num = int(img_name.split('_')[-2])
                    elif position == 'right':
                        label_num = int(img_name.split('_')[-1])
                    label = landmark_label[label_num]
                    bones = data[i]['bones']
                    for j in bones:
                        x,y = bones[j][:2]
                        points.append([x,y])
                    img_dir = img_dir.replace('Z:','DataBase')
                    label_dir = label_dir.replace('Z:','DataBase')

                    landmark = ",".join(map(str,[(int(float(x)), int(float(y))) for x,y in points]))
                    values = (img_name, img_dir, img_format, img_width, img_height, color_space,
                              label_name, label_dir, label_format,
                              label, landmark)

                    value_list2.append(values)
                    if len(value_list2) >= 500:
                        print(value_list2[-1])
                        print(f'{cnt} / {len(label_files)}')
                        doUT.insert_dataset_values(query2_2, value_list2)
                        value_list2 = []
doUT.insert_dataset_values(query1_1, value_list1)
doUT.insert_dataset_values(query2_2, value_list2)
print('done')

('aclass_test_imageID_293_GT_4_5_0', 'DataBase\\sviro\\data\\RGB\\rawdata\\aclass\\aclass_test_imageID_293_GT_4_5_0.png', 'png', 960, 640, 'RGB', 'aclass_test_imageID_293_GT_4_5_0', 'DataBase\\sviro\\data\\RGB\\labeldata\\aclass\\aclass_test_imageID_293_GT_4_5_0.txt', 'txt', 'stuff', 268, 384, 357, 503)
213 / 4976
insert 500 done
('aclass_test_imageID_496_GT_0_2_0', 'DataBase\\sviro\\data\\RGB\\rawdata\\aclass\\aclass_test_imageID_496_GT_0_2_0.png', 'png', 960, 640, 'RGB', 'aclass_test_imageID_496_GT_0_2_0', 'DataBase\\sviro\\data\\RGB\\labeldata\\aclass\\aclass_test_imageID_496_GT_0_2_0.txt', 'txt', 'child', 379, 286, 543, 639)
435 / 4976
insert 500 done
('aclass_train_imageID_1130_GT_1_0_3', 'DataBase\\sviro\\data\\RGB\\rawdata\\aclass\\aclass_train_imageID_1130_GT_1_0_3.png', 'png', 960, 640, 'RGB', 'aclass_train_imageID_1130_GT_1_0_3', 'DataBase\\sviro\\data\\RGB\\labeldata\\aclass\\aclass_train_imageID_1130_GT_1_0_3.txt', 'txt', 'babyseat', 37, 350, 375, 639)
640 / 4976
insert 500

In [68]:
img_dir

'Z:\\sviro\\data\\RGB\\rawdata\\keypoint_gsf\\lexus_train_imageID_290_GT_1_4_0.png'